In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 모델 인스턴스 생성 및 가중치 로드
# model_trained = model # Unet
# model_trained = Unet_resnet.to(device) # Unet + Resnet
# model_trained.load_state_dict(torch.load('./model.pt')) # 가중치파일 경로
model_trained.eval() # 평가 모드로 설정

# 이미지 전처리
def preprocess_image(image_path):
    input_image = Image.open(image_path)
    preprocess = A.Compose(
    [
        A.Normalize(),
        ToTensorV2()
    ])
    input_tensor = preprocess(image=np.array(input_image))['image']
    input_batch = input_tensor.unsqueeze(0) # 배치 차원 추가
    return input_batch

# 이미지 경로
image_path = '/content/drive/MyDrive/인지프/inria/cropped_1024/austin1/images/austin1_0_0.jpg'
gt_path = '/content/drive/MyDrive/인지프/inria/cropped_1024/austin1/binary_masks/austin1_0_0.tif'

# 이미지 전처리 및 예측 수행
with torch.no_grad():
    input_batch = preprocess_image(image_path)
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model_trained.to('cuda')

    output = model_trained(input_batch)

# 결과 처리
output = torch.sigmoid(output).squeeze(0).permute(1,2,0).cpu().numpy() # Sigmoid 적용 및 이미지 형태로 변환
output = (output > 0.5).astype(np.uint8) # Threshold 적용

# 원본 이미지 보기
plt.figure(figsize=(15, 5))  # 전체 그림의 크기를 늘림

original_image = Image.open(image_path)
plt.subplot(1, 3, 1)
plt.imshow(original_image)
plt.axis('off')
plt.title("Original Image")

# ground truth 이미지 보기
gt_image = Image.open(gt_path)
plt.subplot(1, 3, 2)
plt.imshow(gt_image)
plt.axis('off')
plt.title("Ground-Truth Image")

# 결과 이미지 보기 (Matplotlib 사용)
plt.subplot(1, 3, 3)
plt.imshow(output, cmap='gray')  # 'gray'는 흑백 이미지인 경우 사용
plt.axis('off')  # 축 제거
plt.title("Segmentation Output")

plt.show()
